#Group 42 - COMP34812

**Task A :** Natural Language Inference (NLI)

Given a premise and a hypothesis, determine if the hypothesis is true based on the premise. You will be given more than 26K premise-hypothesis pairs as training data, and more than 6K pairs as validation data.

**Solution B :** Deep learning-based approaches that do not employ transformer architectures

*Our final model used an ensemble approach where predictions from four models LSTM, GRU, BiLSTM and BiGRU are combined using soft voting. Pre-trained GloVe embeddings were used. These models were trained on the dataset.*

**Group 42 :** Aisha Wahid & Libby Walton

Libraries

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score

## Preparing Dataset

Cloning resources from Git

In [2]:
!git clone https://github.com/aishawahid/COMP34812.git resources

Cloning into 'resources'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 24 (delta 1), reused 9 (delta 1), pack-reused 15
Receiving objects: 100% (24/24), 41.77 MiB | 23.14 MiB/s, done.
Resolving deltas: 100% (2/2), done.


Loading test data

In [3]:
test_df = pd.read_csv('/content/resources/Data/test.csv')
test_df['hypothesis'] = test_df['hypothesis'].astype(str)

### Tokenizing

In [4]:
# Create a new folder called 'models' in /content/resources
!mkdir -p /content/resources/models

Loading the tokenizer from Google Drive

In [5]:
!gdown --output "/content/resources/models/tokenizer.pkl" 1fRbUz7lCSS0-kqhO7g-U0qP-OOFKYVM_

Downloading...
From: https://drive.google.com/uc?id=1fRbUz7lCSS0-kqhO7g-U0qP-OOFKYVM_
To: /content/resources/models/tokenizer.pkl
100% 1.48M/1.48M [00:00<00:00, 59.5MB/s]


In [6]:
# Load in the tokenizer generated in training from Glove embeddings
with open('/content/resources/models/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [7]:
# Pad and tokeise the premise and hypothesis
test_premise = pad_sequences(tokenizer.texts_to_sequences(np.array(test_df['premise'])), maxlen=84)
test_df['hypothesis'] = test_df['hypothesis'].astype(str)
test_hypothesis = pad_sequences(tokenizer.texts_to_sequences(np.array(test_df['hypothesis'])), maxlen=84)

## Loading Models

Loading models from Google Drive

In [8]:
!gdown --output "/content/resources/models/model_bilstm_70" 1ifQsl5czI_wSXAHaTk85JVM1SWa3xkjB
!gdown --output "/content/resources/models/model_bigru_68" 1uDrjcr8CnmAFkwiwWnopLiz0bePnhrCb
!gdown --output "/content/resources/models/model_lstm_68" 14xUKaj6T-H3lwPaee8ZqRyUTvLqfkeCr
!gdown --output "/content/resources/models/model_gru_65" 1cUNMqLTVaRd44HbYwD7JWfwCdj9Cvdgu

Downloading...
From: https://drive.google.com/uc?id=1ifQsl5czI_wSXAHaTk85JVM1SWa3xkjB
To: /content/resources/models/model_bilstm_70
100% 45.3M/45.3M [00:00<00:00, 68.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uDrjcr8CnmAFkwiwWnopLiz0bePnhrCb
To: /content/resources/models/model_bigru_68
100% 44.8M/44.8M [00:00<00:00, 56.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=14xUKaj6T-H3lwPaee8ZqRyUTvLqfkeCr
To: /content/resources/models/model_lstm_68
100% 44.5M/44.5M [00:00<00:00, 62.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cUNMqLTVaRd44HbYwD7JWfwCdj9Cvdgu
To: /content/resources/models/model_gru_65
100% 44.4M/44.4M [00:00<00:00, 59.6MB/s]


In [9]:
# Load the models - paths need to point to where the models are saved in your Google Drive
with open('/content/resources/models/model_bilstm_70','rb') as f:
    bilstm_model = pickle.load(f)
with open('/content/resources/models/model_bigru_68','rb') as f:
    bigru_model = pickle.load(f)
with open('/content/resources/models/model_lstm_68','rb') as f:
    lstm_model = pickle.load(f)
with open('/content/resources/models/model_gru_65','rb') as f:
    gru_model = pickle.load(f)

Ensemble

In [10]:
accuracies = [0.7006085794864183, 0.6805699866409382, 0.6827964969571025,  0.654890900994508]

# Normalize accuracies to sum up to 1
weights = [accuracy / sum(accuracies) for accuracy in accuracies]

# Aggregate predictions using weighted soft voting
ensemble_predictions = (bilstm_model.predict([test_premise, test_hypothesis]) * weights[0] + bigru_model.predict([test_premise, test_hypothesis]) * weights[1] +
                  lstm_model.predict([test_premise, test_hypothesis]) * weights[2] + gru_model.predict([test_premise, test_hypothesis]) * weights[3])
predicted_labels = ensemble_predictions.argmax(axis=1)

104/104 [==============================] - 7s 58ms/step


Writing predicted labels to csv

In [11]:
pd.set_option('display.max_rows', None)
result_df = pd.DataFrame({'prediction': predicted_labels})
column_name_row = pd.DataFrame({'prediction': ['prediction']}, index=[0])
result_df['prediction'] = result_df['prediction'].astype(int)
result_df = pd.concat([column_name_row, result_df]).reset_index(drop=True)
result_df

,prediction
0,prediction
1,1
2,1
3,1
4,0
5,1
6,0
7,0
8,1
9,1


In [12]:
result_df.to_csv('Group_42_B.csv', encoding='utf-8', index=False, header=False)